In [ ]:
!pip install hvplot

In [9]:
import panel as pn
pn.extension()
import hvplot.pandas
import pandas as pd
import numpy as np

pn.extension(design='material')

csv_file = ("cansat.csv")
data = pd.read_csv(csv_file, parse_dates=["UV"], index_col="UV")

data.tail()


,Time,Temperature,Altitude,Pressure,Orientation,Distance,Deployment,Velocity
UV,,,,,,,,
91,78,18,50,35,N.A,60,Yes,130
74,81,17,40,40,N.A,70,Yes,135
25,84,11,30,45,N.A,80,Yes,140
91,87,21,20,50,N.A,90,Yes,145
40,90,11,0,55,N.A,100,Yes,150


In [10]:
def transform_data(variable, window, sigma):
    ''' Calculates the rolling average and the outliers '''
    avg = data[variable].rolling(window=window).mean()
    residual = data[variable] - avg
    std = residual.rolling(window=window).std()
    outliers = np.abs(residual) > std * sigma
    return avg, avg[outliers]

def create_plot(variable="Temperature", window=30, sigma=10):
    ''' Plots the rolling average and the outliers '''
    avg, highlight = transform_data(variable, window, sigma)
    return avg.hvplot(height=300, width=400, legend=False) * highlight.hvplot.scatter(
        color="orange", padding=0.1, legend=False
    )

In [11]:
create_plot(variable='Temperature', window=20, sigma=10)

:Overlay
   .Curve.Temperature   :Curve   [UV]   (Temperature)
   .Scatter.Temperature :Scatter   [UV]   (Temperature)

In [12]:
variable_widget = pn.widgets.Select(name="variable", value="Temperature", options=list(data.columns))
window_widget = pn.widgets.IntSlider(name="window", value=30, start=1, end=60)
sigma_widget = pn.widgets.IntSlider(name="sigma", value=10, start=0, end=20)

In [13]:
bound_plot = pn.bind(create_plot, variable=variable_widget, window=window_widget, sigma=sigma_widget)

In [14]:
first_app = pn.Column(variable_widget, window_widget, sigma_widget, bound_plot)

first_app.servable()

Column(design=<class 'panel.theme.materi...)
    [0] Select(design=<class 'panel.theme.materi..., name='variable', options=['Time', 'Temperature', ...], value='Temperature')
    [1] IntSlider(design=<class 'panel.theme.materi..., end=60, name='window', start=1, value=30)
    [2] IntSlider(design=<class 'panel.theme.materi..., end=20, name='sigma', value=10)
    [3] ParamFunction(function, _pane=HoloViews, defer_load=False, design=<class 'panel.theme.materi...)